This exercise will require you to pull some data from the Qunadl API. Qaundl is currently the most widely used aggregator of financial market data.

As a first step, you will need to register a free account on the http://www.quandl.com website.

After you register, you will be provided with a unique API key, that you should store:

In [1]:
# Store the API key as a string - according to PEP8, constants are always named in all upper case
API_KEY = ''

Qaundl has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Quandl API instructions here: https://docs.quandl.com/docs/time-series

While there is a dedicated Python package for connecting to the Quandl API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [3]:
# First, import the relevant modules
import requests

In [6]:
# Now, call the Quandl API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned
# Now, call the Quandl API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned
URL = 'https://www.quandl.com/api/v3/datasets/FSE/AFX_X.json'

#start/end date
URL = URL + '?start_date=2018-01-01&end_date=2018-01-02'

#use key
URL = URL + '&api_key=' + API_KEY#create response object
#create response object
response = requests.get(URL)
#create a json from the response object.
CZM = response.json()

In [7]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure
print(CZM)

{'dataset': {'dataset_code': 'AFX_X', 'column_index': None, 'premium': False, 'id': 10095370, 'end_date': '2018-01-02', 'type': 'Time Series', 'oldest_available_date': '2000-06-07', 'database_code': 'FSE', 'newest_available_date': '2018-10-30', 'order': None, 'name': 'Carl Zeiss Meditec (AFX_X)', 'limit': None, 'description': 'Stock Prices for Carl Zeiss Meditec (AFX) from the Frankfurt Stock Exchange.<br><br>Trading System: Xetra<br><br>ISIN: DE0005313704', 'refreshed_at': '2018-10-30T22:30:47.066Z', 'column_names': ['Date', 'Open', 'High', 'Low', 'Close', 'Change', 'Traded Volume', 'Turnover', 'Last Price of the Day', 'Daily Traded Units', 'Daily Turnover'], 'database_id': 6129, 'frequency': 'daily', 'transform': None, 'collapse': None, 'start_date': '2018-01-01', 'data': [['2018-01-02', 52.05, 52.4, 51.2, 51.4, None, 54435.0, 2807533.0, None, None, None]]}}


These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

In [8]:
#1)Collect data from the Franfurt Stock Exchange,
#  for the ticker AFX_X, for the whole year 2017 
#  (keep in mind that the date format is YYYY-MM-DD).
URL = 'https://www.quandl.com/api/v3/datasets/FSE/AFX_X.json'
URL = URL + '?start_date=2017-01-01&end_date=2017-12-31'
URL = URL + '&api_key=' + API_KEY
response = requests.get(URL)

#2)Convert the returned JSON object into a Python dictionary.
CZM_json = response.json()
CZM_dict = {item[0]:item[1:] for item in CZM_json['dataset']['data']}

In [71]:
#3-6) See answers below 

#Function to find the minimum while in a for loop 
def min_fuction(new_value,previous_value):
    if new_value is not None:
        if new_value < previous_value:
            previous_value = float(new_value)  
    return previous_value 

#Function to find the maximum while in a for loop 
def max_fuction(new_value,previous_value):
    if new_value is not None:
        if new_value > previous_value:
            previous_value = float(new_value)  
    return previous_value 

#Function to find the highest daily while in a for loops 
def daily_function(low,high,previous_value):    
    if (high is not None) and (low is not None): 
        if high-low > previous_value:
            previous_value = high-low  
    return previous_value

#Function to find the highest difference in closing of two consecutive days while in a for loop 
def consecutive_diff(value,previous_value,previous_high):
    if previous_value is not None:
        if abs( value - previous_value) > abs(previous_high):
            previous_high = value - previous_value    
    previous_value = value
    return (previous_value,previous_high)
        
#Function to find the median of a column of a dictionary of lists.
def median_column(dictionary,column):
    volume_list = sorted([value[column] for key,value in dictionary.items()])
    median = volume_list[len(volume_list)//2] #if lens of list is odd
    if len(volume_list)%2 == 0: #if lens of list is even
        median = (volume_list[len(volume_list)//2-1] + volume_list[len(volume_list)//2])/2
    return median

def main_seq():
    
    #Creates labels for every column of dictionary
    CZM_label = CZM_json['dataset']['column_names'].copy() 
    CZM_label.remove('Date')
    open_index = CZM_label.index('Open') 
    high_index = CZM_label.index('High')
    low_index   = CZM_label.index('Low')
    close_index = CZM_label.index('Close')
    volume_index = CZM_label.index('Traded Volume')

    #Create variables to be reported at end with dummy values
    max_open = 0.0
    min_open = 999999999.0
    daily_change = 0.0
    close_change = 0.0
    volume_avg = 0.0

    for index,(key,value) in enumerate(sorted(CZM_dict.items())):

        #find min 
        min_open=min_fuction(value[open_index],min_open)

        #find max
        max_open=max_fuction(value[open_index],max_open)

        #find daily change
        daily_change=daily_function(value[low_index],value[high_index],daily_change)

        #find change between two consecutive days
        if index == 0: 
            previous_close = value[close_index]
        else:
            previous_close,close_change = consecutive_diff(value[close_index],previous_close,close_change)

        if (value[volume_index] is not None) :
            #sum over traded volume which will be used to find average
            volume_avg += value[volume_index]

    #3) Calculate what the highest and lowest opening prices were for the stock in this period.
    print('min = {:.2f} max = {:.2f}'.format(min_open,max_open))

    #4) What was the largest change in any one day (based on High and Low price)?
    print('max change = {:.2f}'.format(daily_change))

    # 5)What was the largest change between any two days (based on Closing Price)?
    print('largest change between any two days = {:.2f}'.format(close_change))

    #6)What was the average daily trading volume during this year?
    volume_avg /= float(len(CZM_dict))
    print('average trading volume = {:.2f}'.format(volume_avg))

    #7)What was the median trading volume during this year.
    print('median volume traded = {:.2f}'.format(median_column(CZM_dict,volume_index)))

if __name__ == '__main__':
    main_seq()

min = 34.00 max = 53.11
max change = 2.81
largest change between any two days = -2.56
average trading volume = 89124.34
median volume traded = 76286.00
